# UK Map of Confirmed Cases

In [10]:
import numpy as np                    
import pandas as pd                  
import plotly.graph_objects as go    
import plotly.io as pio              
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from IPython.display import display, clear_output

pio.templates['shahin'] = pio.to_templated(go.Figure().update_layout(legend=dict(orientation="h",y=1.1, x=.5, xanchor='center'),margin=dict(t=0,r=0,b=0,l=0))).layout.template
pio.templates.default = 'shahin'

Table with the locations, confirmed cases and latitude and longitude columns that are empty, but will be populated.

In [11]:
data = pd.read_csv('https://shahinrostami.com/datasets/phe_covid_uk.csv')
data.head()

,local_authority,confirmed_cases,lat,lon
0,Barking and Dagenham,1,NaN,NaN
1,Barnet,8,NaN,NaN
2,Barnsley,2,NaN,NaN
3,Bath and North East Somerset,0,NaN,NaN
4,Bedford,0,NaN,NaN


In [12]:
geolocator = Nominatim(user_agent = "covid_shahinrostami.com")

In [13]:
data.local_authority[10]

'Bournemouth, Christchurch and Poole'

In [14]:
location = geolocator.geocode(f"{data.local_authority[10]}, UK")
location

Location(Bournemouth, South West England, England, BH1 2BU, United Kingdom, (50.7194784, -1.8767712, 0.0))

In [15]:
print(location.latitude, location.longitude)

50.7194784 -1.8767712


Populating every latitude and longitude values.

In [16]:
for index, row in data.iterrows():
    location = geolocator.geocode(f"{row.local_authority}, UK",timeout=100)

    data.loc[index,'lat'] = location.latitude 
    data.loc[index,'lon'] = location.longitude

    clear_output(wait = True)
    amount_unloaded = np.floor(((data.shape[0]-index)/data.shape[0])*25).astype(int)
    amount_loaded = np.ceil((index/data.shape[0])*25).astype(int)
    loading = f"Retrieving locations >{'|'*amount_loaded}{'.'*amount_unloaded}<"
    display(loading)

print("Done!")

'Retrieving locations >|||||||||||||||||||||||||<'

Done!


# UK bubble map of confirmed cases 

The larger bubbles indicate more confirmed cases.

In [17]:
data['text'] = data['local_authority'] + '<br>Confirmed Cases ' + (data['confirmed_cases']).astype(str)
        
fig = go.Figure()

fig.add_trace(go.Scattergeo(
    locationmode = 'country names',
    lon = data['lon'],
    lat = data['lat'],
    text = data['text'],
    marker = dict(
        size = data['confirmed_cases']/.1,
        color = 'rgb(200,0,0)',
        line_color = 'rgb(122,0,0)',
        line_width = 0.5,
        sizemode = 'area')
))

fig.update_layout(
    geo = dict(
        resolution = 50,
        scope = 'europe',
        center = {'lat': (data.lat.min() + data.lat.max())/2,
                'lon': (data.lon.min() + data.lon.max())/2},
        projection = go.layout.geo.Projection(
            type = 'azimuthal equal area',
            scale = 8
        ),
        landcolor = 'rgb(217, 217, 217)',
        showocean = True,
    )
)

fig.show()